In [1]:
import torch
import torch.nn as nn
import time
import argparse

import os
import datetime

from torch.distributions.categorical import Categorical

# visualization 
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

device = torch.device("cpu"); gpu_id = -1 # select CPU

gpu_id = '0' # select a single GPU  
#gpu_id = '2,3' # select multiple GPUs  
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)  
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU name: {:s}, gpu_id: {:s}'.format(torch.cuda.get_device_name(0),gpu_id))   
    
print(device)
print('pytorch version = ',torch.__version__)

GPU name: NVIDIA GeForce RTX 2080 Ti, gpu_id: 0
cuda
pytorch version =  1.10.0


In [2]:
import math
import numpy as np
import torch.nn.functional as F
import random
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
class TransEncoderNet(nn.Module):
    """
    Encoder network based on self-attention transformer
    Inputs :  
      h of size      (bsz, nb_nodes, dim_emb)    batch of input cities
    Outputs :  
      h of size      (bsz, nb_nodes, dim_emb)    batch of encoded cities
      score of size  (bsz, nb_nodes, nb_nodes+1) batch of attention scores
    """
    
    def __init__(self, nb_layers, dim_emb, nb_heads, dim_ff, batchnorm):
        super(TransEncoderNet, self).__init__()
        assert dim_emb == nb_heads* (dim_emb//nb_heads) # check if dim_emb is divisible by nb_heads
        self.MHA_layers = nn.ModuleList( [nn.MultiheadAttention(dim_emb, nb_heads) for _ in range(nb_layers)] )
        self.linear1_layers = nn.ModuleList( [nn.Linear(dim_emb, dim_ff) for _ in range(nb_layers)] )
        self.linear2_layers = nn.ModuleList( [nn.Linear(dim_ff, dim_emb) for _ in range(nb_layers)] )   
        if batchnorm:
            self.norm1_layers = nn.ModuleList( [nn.BatchNorm1d(dim_emb) for _ in range(nb_layers)] )
            self.norm2_layers = nn.ModuleList( [nn.BatchNorm1d(dim_emb) for _ in range(nb_layers)] )
        else:
            self.norm1_layers = nn.ModuleList( [nn.LayerNorm(dim_emb) for _ in range(nb_layers)] )
            self.norm2_layers = nn.ModuleList( [nn.LayerNorm(dim_emb) for _ in range(nb_layers)] )
        self.nb_layers = nb_layers
        self.nb_heads = nb_heads
        self.batchnorm = batchnorm
        
    def forward(self, h):      
        # PyTorch nn.MultiheadAttention requires input size (seq_len, bsz, dim_emb) 
        h = h.transpose(0,1) # size(h)=(nb_nodes, bsz, dim_emb)  
        # L layers
        for i in range(self.nb_layers):
            h_rc = h # residual connection, size(h_rc)=(nb_nodes, bsz, dim_emb)
            h, score = self.MHA_layers[i](h, h, h) # size(h)=(nb_nodes, bsz, dim_emb), size(score)=(bsz, nb_nodes, nb_nodes)
            # add residual connection
            
            h = h_rc + h # size(h)=(nb_nodes, bsz, dim_emb)
            if self.batchnorm:
                # Pytorch nn.BatchNorm1d requires input size (bsz, dim, seq_len)
                h = h.permute(1,2,0).contiguous() # size(h)=(bsz, dim_emb, nb_nodes)
                h = self.norm1_layers[i](h)       # size(h)=(bsz, dim_emb, nb_nodes)
                h = h.permute(2,0,1).contiguous() # size(h)=(nb_nodes, bsz, dim_emb)
            else:
                h = self.norm1_layers[i](h)       # size(h)=(nb_nodes, bsz, dim_emb) 
            # feedforward
            h_rc = h # residual connection
            h = self.linear2_layers[i](torch.relu(self.linear1_layers[i](h)))
            h = h_rc + h # size(h)=(nb_nodes, bsz, dim_emb)
            if self.batchnorm:
                h = h.permute(1,2,0).contiguous() # size(h)=(bsz, dim_emb, nb_nodes)
                h = self.norm2_layers[i](h)       # size(h)=(bsz, dim_emb, nb_nodes)
                h = h.permute(2,0,1).contiguous() # size(h)=(nb_nodes, bsz, dim_emb)
            else:
                h = self.norm2_layers[i](h) # size(h)=(nb_nodes, bsz, dim_emb)
        # Transpose h
        h = h.transpose(0,1) # size(h)=(bsz, nb_nodes, dim_emb)
        return h, score
    

class Attention(nn.Module):
    def __init__(self, n_hidden):
        super(Attention, self).__init__()
        self.size = 0
        self.batch_size = 0
        self.dim = n_hidden
        
        v  = torch.FloatTensor(n_hidden)
        self.v  = nn.Parameter(v)
        self.v.data.uniform_(-1/math.sqrt(n_hidden), 1/math.sqrt(n_hidden))
        
        # parameters for pointer attention
        self.Wref = nn.Linear(n_hidden, n_hidden)
        self.Wq = nn.Linear(n_hidden, n_hidden)
    
    
    def forward(self, q, ref):       # query and reference
        self.batch_size = q.size(0)
        self.size = int(ref.size(0) / self.batch_size)
        q = self.Wq(q)     # (B, dim)
        ref = self.Wref(ref)
        ref = ref.view(self.batch_size, self.size, self.dim)  # (B, size, dim)
        
        q_ex = q.unsqueeze(1).repeat(1, self.size, 1) # (B, size, dim)
        # v_view: (B, dim, 1)
        v_view = self.v.unsqueeze(0).expand(self.batch_size, self.dim).unsqueeze(2)
        
        # (B, size, dim) * (B, dim, 1)
        u = torch.bmm(torch.tanh(q_ex + ref), v_view).squeeze(2)
        
        return u, ref
    
class LSTM(nn.Module):
    def __init__(self, n_hidden):
        super(LSTM, self).__init__()
        
        # parameters for input gate
        self.Wxi = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Whi = nn.Linear(n_hidden, n_hidden)    # W(ht)
        self.wci = nn.Linear(n_hidden, n_hidden)    # w(ct)
        
        # parameters for forget gate
        self.Wxf = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Whf = nn.Linear(n_hidden, n_hidden)    # W(ht)
        self.wcf = nn.Linear(n_hidden, n_hidden)    # w(ct)
        
        # parameters for cell gate
        self.Wxc = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Whc = nn.Linear(n_hidden, n_hidden)    # W(ht)
        
        # parameters for forget gate
        self.Wxo = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Who = nn.Linear(n_hidden, n_hidden)    # W(ht)
        self.wco = nn.Linear(n_hidden, n_hidden)    # w(ct)
    
    
    def forward(self, x, h, c):       # query and reference
        
        # input gate
        i = torch.sigmoid(self.Wxi(x) + self.Whi(h) + self.wci(c))
        # forget gate
        f = torch.sigmoid(self.Wxf(x) + self.Whf(h) + self.wcf(c))
        # cell gate
        c = f * c + i * torch.tanh(self.Wxc(x) + self.Whc(h))
        # output gate
        o = torch.sigmoid(self.Wxo(x) + self.Who(h) + self.wco(c))
        
        h = o * torch.tanh(c)
        
        return h, c

class HPN(nn.Module):
    def __init__(self, n_feature, n_hidden):

        super(HPN, self).__init__()
        self.city_size = 0
        self.batch_size = 0
        self.dim = n_hidden
        
        # lstm for first turn
        #self.lstm0 = nn.LSTM(n_hidden, n_hidden)main_training - Jupyter Notebook
        
        # pointer layer
        self.pointer = Attention(n_hidden)
        self.TransPointer = Attention(n_hidden)
        
        # lstm encoder
        self.encoder = LSTM(n_hidden)
        
        # trainable first hidden input
        h0 = torch.FloatTensor(n_hidden)
        c0 = torch.FloatTensor(n_hidden)
        # trainable latent variable coefficient
        print('here') 
        alpha = torch.ones(1)
        self.h0 = nn.Parameter(h0)
        self.c0 = nn.Parameter(c0)
        
        self.alpha = nn.Parameter(alpha)
        self.h0.data.uniform_(-1/math.sqrt(n_hidden), 1/math.sqrt(n_hidden))
        self.c0.data.uniform_(-1/math.sqrt(n_hidden), 1/math.sqrt(n_hidden))
        
        r1 = torch.ones(1)
        r2 = torch.ones(1)
        r3 = torch.ones(1)
        self.r1 = nn.Parameter(r1)
        self.r2 = nn.Parameter(r2)
        self.r3 = nn.Parameter(r3)
        
        # embedding
        self.embedding_x = nn.Linear(n_feature, n_hidden)
        self.embedding_all = nn.Linear(n_feature, n_hidden)
        self.Transembedding_all = TransEncoderNet(6, 128, 8, 512, batchnorm=True)#6,128,8,512
        
        # vector to start decoding 
        self.start_placeholder = nn.Parameter(torch.randn(n_hidden))
        
        # weights for GNN
        self.W1 = nn.Linear(n_hidden, n_hidden)
        self.W2 = nn.Linear(n_hidden, n_hidden)
        self.W3 = nn.Linear(n_hidden, n_hidden)
        
        # aggregation function for GNN
        self.agg_1 = nn.Linear(n_hidden, n_hidden)
        self.agg_2 = nn.Linear(n_hidden, n_hidden)
        self.agg_3 = nn.Linear(n_hidden, n_hidden)
    
    
    def forward(self,context,Transcontext, x, X_all, mask, mask_temp, h=None, c=None, latent=None):
        '''
        Inputs (B: batch size, size: city size, dim: hidden dimension)
        
        x: current city coordinate (B, 2)
        X_all: all cities' cooridnates (B, size, 2)
        mask: mask visited cities
        h: hidden variable (B, dim)
        c: cell gate (B, dim)
        latent: latent pointer vector from previous layer (B, size, dim)
        
        Outputs
        
        softmax: probability distribution of next city (B, size)
        h: hidden variable (B, dim)
        c: cell gate (B, dim)
        latent_u: latent pointer vector for next layer
        '''
        
        self.batch_size = X_all.size(0)
        self.city_size = X_all.size(1)
        
        # Check if this the first iteration loop
        if h is None or c is None:
            x          = self.start_placeholder  
            context = self.embedding_all(X_all)
            Transcontext,_ = self.Transembedding_all(context)
            
            # =============================
            # graph neural network encoder
            # =============================

            # (B, size, dim)
            context = context.reshape(-1, self.dim)
            Transcontext = Transcontext.reshape(-1, self.dim)

            context = self.r1 * self.W1(context)\
                + (1-self.r1) * F.relu(self.agg_1(context/(self.city_size-1)))

            context = self.r2 * self.W2(context)\
                + (1-self.r2) * F.relu(self.agg_2(context/(self.city_size-1)))

            context = self.r3 * self.W3(context)\
                + (1-self.r3) * F.relu(self.agg_3(context/(self.city_size-1)))
            h0 = self.h0.unsqueeze(0).expand(self.batch_size, self.dim)
            c0 = self.c0.unsqueeze(0).expand(self.batch_size, self.dim)

            h0 = h0.unsqueeze(0).contiguous()
            c0 = c0.unsqueeze(0).contiguous()
            
            # let h0, c0 be the hidden variable of first turn
            h = h0.squeeze(0)
            c = c0.squeeze(0)
        else:
            x          = self.embedding_x(x)
        # LSTM encoder1900/2500
        h, c = self.encoder(x, h, c)
        # query vector
        q = h
        # pointer
        u1, _ = self.pointer(q, context)
        u2 ,_ = self.TransPointer(q, Transcontext)
        # Avg Agg between the two attention vector
        u = torch.maximum(u1,u2)
        latent_u = u.clone()
                
        u = 10 * torch.tanh(u) + mask
        u = u + mask_temp
        return context, Transcontext,F.softmax(u, dim=1), h, c, latent_u
# params= list(HPN(n_feature = 2, n_hidden = 128).parameters())
# print(len(params))
# for name,parameters in HPN(n_feature = 2, n_hidden = 128).named_parameters():
#     print(name,':',parameters.size())

# daniel rectangle feature handle

In [3]:
'''
This part I designed the rectangle-characterized TSP, that means for every step the agent walk through a corner,
then he travel through the whole rectangle using zig-zag, finally he ends up at one of the rest corners of 
the rextangle, so, it equals the agent walk through three points at one step, in practice, I add three points into 
mask to make them unselectable.
'''
def rectangle_process(temp,idx,Y,Y0,mask,k,B,Not_choosen_ones,Not_last_time,mask_list_num):
    Y1 = Y[zero_to_bsz, idx.data].clone()
    rectangle_inf = idx/4
    feature_table = temp.outcorner_getout(rectangle_inf,B,mask_list_num)
    feature_table = torch.Tensor(feature_table).type(torch.long)
    feature_table = feature_table.cuda()
    Y_corner = Y[zero_to_bsz, feature_table[:,0].data].clone()
    if k ==0:
        reward = torch.zeros(B).cuda()
    if k > 0:
        reward = torch.sum((Y1[:,(0,1)] - Y0[:,(0,1)])**2 , dim=1 )**0.5
        reward += torch.sum((Y_corner[:,(0,1)] - Y1[:,(0,1)])**2 , dim=1 )**0.5
        #dis = (Y1 - Y0)**2
        #dis_1 = (Y_corner - Y1)**2
        #reward = torch.maximum(dis[:,0],dis[:,1])**0.5
        #reward += torch.maximum(dis_1[:,0],dis_1[:,1])**0.5  
    mask[zero_to_bsz, idx.data] += -np.inf
    #這邊是全都是四邊形時的配置
    mask[zero_to_bsz, feature_table[:,0].data] += -np.inf
    mask[zero_to_bsz, feature_table[:,1].data] += -np.inf    
    mask[zero_to_bsz, feature_table[:,2].data] += -np.inf 
    if_frozen = Not_last_time[zero_to_bsz, idx.data]
    Not_choosen_ones[zero_to_bsz,if_frozen * (idx.data +4)] = 0
    if_frozen = Not_last_time[zero_to_bsz, feature_table[:,0].data]
    Not_choosen_ones[zero_to_bsz,if_frozen * (feature_table[:,0] +4)] = 0
    if_frozen = Not_last_time[zero_to_bsz, feature_table[:,1].data]
    Not_choosen_ones[zero_to_bsz,if_frozen * (feature_table[:,1] +4)] = 0  
    if_frozen = Not_last_time[zero_to_bsz, feature_table[:,2].data]
    Not_choosen_ones[zero_to_bsz,if_frozen * (feature_table[:,2] +4)] = 0
    Not_choosen_ones[zero_to_bsz,0] = 1
    return reward, Y_corner, Y_corner
def rectangle_process_actor(temp,idx,Y,Y0,mask,k,B,i,path_gazebo,Not_choosen_ones,Not_last_time,mask_list_num):
    Y1 = Y[zero_to_bsz, idx.data].clone()
    rectangle_inf = idx/4
    feature_table = temp.outcorner_getout(rectangle_inf,B,mask_list_num)
    feature_table = torch.Tensor(feature_table).type(torch.long)
    feature_table = feature_table.cuda()
    Y_corner = Y[zero_to_bsz, feature_table[:,0].data].clone()
    if i % 100 == 0:
        path_gazebo.append([idx.data[0].tolist(),feature_table[:,0].data[0].tolist()])
    if k ==0:
        reward = torch.zeros(B).cuda()
    if k > 0:
        reward = torch.sum((Y1[:,(0,1)] - Y0[:,(0,1)])**2 , dim=1 )**0.5
        reward += torch.sum((Y_corner[:,(0,1)] - Y1[:,(0,1)])**2 , dim=1 )**0.5
        #dis = (Y1 - Y0)**2
        #dis_1 = (Y_corner - Y1)**2
        #reward = torch.maximum(dis[:,0],dis[:,1])**0.5
        #reward += torch.maximum(dis_1[:,0],dis_1[:,1])**0.5  
    mask[zero_to_bsz, idx.data] += -np.inf
    mask[zero_to_bsz, feature_table[:,0].data] += -np.inf
    mask[zero_to_bsz, feature_table[:,1].data ] += -np.inf    
    mask[zero_to_bsz, feature_table[:,2].data ] += -np.inf
    if_frozen = Not_last_time[zero_to_bsz, idx.data]
    next_time_visited = if_frozen * (idx.data +4)
    Not_choosen_ones[zero_to_bsz,next_time_visited] = 0
    if_frozen = Not_last_time[zero_to_bsz, feature_table[:,0].data]
    next_time_visited = if_frozen * (feature_table[:,0].data+4)
    Not_choosen_ones[zero_to_bsz,next_time_visited] = 0
    if_frozen = Not_last_time[zero_to_bsz, feature_table[:,1].data]
    next_time_visited = if_frozen * (feature_table[:,1].data+4)
    Not_choosen_ones[zero_to_bsz,next_time_visited] = 0  
    if_frozen = Not_last_time[zero_to_bsz, feature_table[:,2].data]
    next_time_visited = if_frozen * (feature_table[:,2].data+4)
    Not_choosen_ones[zero_to_bsz,next_time_visited] = 0
    Not_choosen_ones[zero_to_bsz,0] = 1
    return reward, Y_corner, Y_corner

In [4]:
def pos(lst):
    return [x for x in lst if x > 0] or None
def unmask_checker(R,B, waiting_time_list, mask_temp):
    R_reshape = R.view(B,1)
    if_dried = waiting_time_list - R_reshape
    waiting_time_remain = waiting_time_list - R_reshape
    
    if_dried = if_dried<0
    # if a ROI is ready for re-opened, it would be True in if_dried matrix
    mask_temp[if_dried] = 0
    return mask_temp,waiting_time_remain

In [ ]:
import yaml
from torch.distributions.categorical import Categorical
# visualization
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm_notebook
import torch.nn.functional as F
####### my own import file ##########
from listofpathpoint import input_handler
import cnc_input
#from hybrid_models import HPN
####### my own import file ##########
'''
so, the models we have are TransEncoderNet,
                            Attention
                            LSTM
                            HPN
each one have initial parameters and the forward part, 
once we have the forward part, the back propagation will 
finished automatically by pytorch  
'''
TOL = 1e-3
TINY = 1e-15
learning_rate = 1e-4   #learning rate
B = 128            #batch size
B_valLoop = 20
steps = 500
n_epoch = 100       # epochs
map_number = 0
record_actor = []
record_critic = []
dimension = 3
speed_of_nozzle = 30
punish_of_waiting = 10000/speed_of_nozzle
range_of_wait = 10

print('======================')
print('prepare to train')
print('======================')
print('Hyper parameters:')
print('learning rate', learning_rate)
print('batch size', B)
print('steps', steps)
print('epoch', n_epoch)
print('======================')

'''
instantiate a training network and a baseline network
'''


try:
    del Actor  # remove existing model
    del Critic # remove existing model
except:
    pass
Actor = HPN(n_feature = dimension, n_hidden = 128)
Critic = HPN(n_feature = dimension, n_hidden = 128)
optimizer = optim.Adam(Actor.parameters(), lr=learning_rate)

# Putting Critic model on the eval mode
Actor = Actor.to(device)
Critic = Critic.to(device)
Critic.eval()

epoch_ckpt = 0
tot_time_ckpt = 0

val_mean = []
val_std = []
maplist = ['10&15data/25_chips/25_1.json'
          ,'10&15data/25_chips/25_1.json','10&15data/25_chips/25_1.json'\
          ,'10&15data/25_chips/25_1.json','10&15data/25_chips/25_1.json','10&15data/25_chips/25_1.json'
          ]
          
plot_performance_train = []
plot_performance_baseline = []
# recording the result of the resent epoch makes it available for future
#*********************# Uncomment these lines to load the previous check point
"""
checkpoint_file = "checkpoint/mutimap_20.pkl"
checkpoint = torch.load(checkpoint_file, map_location=device)
epoch_ckpt = checkpoint['epoch'] + 1
tot_time_ckpt = checkpoint['tot_time']
plot_performance_train = checkpoint['plot_performance_train']
plot_performance_baseline = checkpoint['plot_performance_baseline']
Critic.load_state_dict(checkpoint['model_baseline'])
Actor.load_state_dict(checkpoint['model_train'])
optimizer.load_state_dict(checkpoint['optimizer'])

print('Re-start training with saved checkpoint file={:s}\n  Checkpoint at epoch= {:d} and time={:.3f}min\n'.format(checkpoint_file,epoch_ckpt-1,tot_time_ckpt/60))
"""

#***********************# Uncomment these lines to load the previous check point

# Main training loop
# The core training concept mainly upon Sampling from the actor
# then taking the greedy action from the critic

start_training_time = time.time()
time_stamp = datetime.datetime.now().strftime("%y-%m-%d--%H-%M-%S") # Load the time stamp

C = 0       # baseline => the object which the actor can compare
R = 0       # reward

temp = input_handler(maplist[map_number])
X_temp, mask_list_num, waiting_time_list = temp.baseline_points()
#change the waiting time to the distance that we can measure
mask_list_num_with_lastone = mask_list_num.copy()
size = mask_list_num.pop()*4
Not_last_time = torch.ones(B,size, dtype=torch.bool).cuda()
zero_to_bsz = torch.arange(B, device = device) # a list contains 0 to (batch size -1)
zero_to_size = torch.arange(size, device = device)
for last_one in mask_list_num:
    Not_last_time[zero_to_bsz,(last_one-1)*4] = False
    Not_last_time[zero_to_bsz,(last_one-1)*4+1] = False
    Not_last_time[zero_to_bsz,(last_one-1)*4+2] = False
    Not_last_time[zero_to_bsz,(last_one-1)*4+3] = False

for corner in X_temp:
    corner[dimension -1] *= speed_of_nozzle

size_rec = int(size/4)
print(size_rec)

X_temp = torch.FloatTensor(X_temp)
waiting_time_list = torch.FloatTensor(waiting_time_list)
X = X_temp.repeat(B,1,1)

waiting_time_list = waiting_time_list.repeat(B,1)
waiting_time_list = waiting_time_list.cuda()
waiting_time_list *= speed_of_nozzle
waiting_time_initial = waiting_time_list.clone()
X = X.cuda()
waiting_time_list *= speed_of_nozzle

for epoch in range(0, n_epoch):
    # re-start training with saved checkpoint
    epoch += epoch_ckpt # adding the number of the former epochs
    # Train the model for one epoch
    start = time.time() # record the starting time
    Actor.train() 
    path_gazebo = []
    for i in range(1, steps+1): # 1 ~ 2500 steps
        # mask some points that are not the first visited points
        R = torch.zeros(B).cuda()
        logprobs = 0
        reward = torch.zeros(B).cuda()
        mask = torch.zeros(B,size).cuda() # use mask to make some points impossible to choose
        mask_temp = torch.zeros(B,size).cuda() # mask_temp for temp-masked points(wait for drying)
        mask_temp += -np.inf
        for initial_area_idx in mask_list_num:
            mask_temp[zero_to_bsz,initial_area_idx*4] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +1] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +2] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +3] = 0
        Not_choosen_ones = torch.ones(B,size).cuda()
        x = torch.zeros(B,dimension)# Y[:,0,:] #set the first point to x
        h = None
        c = None
        context = None        # set Y_ini to the out corner
        Transcontext = None
        Y0 = None
        waiting_time_list = waiting_time_initial
        # Actor Sampling phase
        for k in range(size_rec):
            #add the time passed to the time axis of nessesary points
            reward_reshape = reward.view(B,1)
            waiting_time_list[zero_to_bsz] += Not_choosen_ones[zero_to_bsz] * reward_reshape
            Y = X.view(B,size,dimension)
            #refresh the mask list, fits the present situation
            while(True):
                mask_temp,waiting_time_remain = unmask_checker(R,B, waiting_time_list, mask_temp)
                #check if all the points to be masked,if so, raise the total reward R, and check again
                test_mask_tool = torch.zeros(B,size).cuda()
                test_mask_tool += mask
                test_mask_tool += mask_temp
                test_output = F.softmax(test_mask_tool, dim=1)
                test_anyone_open = torch.sum(test_output,1)
                test_anyone_open = test_anyone_open>0
                test_no_one_open = torch.logical_not(test_anyone_open)
                if torch.sum(test_no_one_open.type(torch.uint8)) >0:
                    R+=test_no_one_open*waiting_time_remain
                    continue
                break
            context, Transcontext, output, h, c, _ = Actor(context,Transcontext,x=x, X_all=X,h=h, c=c, mask=mask,mask_temp=mask_temp)
            sampler = torch.distributions.Categorical(output)
            idx = sampler.sample()
            #prepare for the back propagation of pytorch
            reward, Y0,x = rectangle_process_actor(temp, idx,Y,Y0,mask,k,B,i,path_gazebo,
                                                Not_choosen_ones,Not_last_time,mask_list_num_with_lastone)
            R += reward
            logprobs += torch.log(output[zero_to_bsz, idx.data] + TINY)
        print(i)
# critic baseline phase, use the baseline to compute the actual reward of agent at that time
        mask = torch.zeros(B,size).cuda() # use mask to make some points impossible to choose
        C = torch.zeros(B).cuda()
        baseline = torch.zeros(B).cuda()
        mask_temp = torch.zeros(B,size).cuda() # mask_temp for temp-masked points(wait for drying)
        mask_temp += -np.inf
        for initial_area_idx in mask_list_num:
            mask_temp[zero_to_bsz,initial_area_idx*4] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +1] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +2] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +3] = 0
        Not_choosen_ones = torch.ones(B,size).cuda()
        x = torch.zeros(B,dimension)#Y[:,0,:] #set the first point to x
        h = None
        c = None
        context = None
        Transcontext = None
        C0 = None
        waiting_time_list = waiting_time_initial
        # compute tours for baseline without grad "Cause we want to fix the weights for the critic"
        with torch.no_grad():
            for k in range(size_rec):     
                baseline_reshape = baseline.view(B,1)
                waiting_time_list[zero_to_bsz] += Not_choosen_ones[zero_to_bsz] * baseline_reshape
                Y = X.view(B,size,dimension)
                while(True):
                    mask_temp = unmask_checker(C,B, waiting_time_list, mask_temp)
                    #check if all the points to be masked,if so, raise the total reward R, and check again
                    test_mask_tool = torch.zeros(B,size).cuda()
                    test_mask_tool += mask
                    test_mask_tool += mask_temp
                    test_output = F.softmax(test_mask_tool, dim=1)
                    test_anyone_open = torch.sum(test_output,1)
                    test_anyone_open = test_anyone_open>0
                    test_no_one_open = torch.logical_not(test_anyone_open)
                    if torch.sum(test_no_one_open.type(torch.uint8)) >0:
                        C+=test_no_one_open*(speed_of_nozzle*range_of_wait)
                        continue
                    break
                context, Transcontext, output, h, c, _ = Critic(context,Transcontext,x=x, X_all=X,h=h, c=c, mask=mask,mask_temp=mask_temp)
                idx = torch.argmax(output, dim=1) # ----> greedy baseline critic
                # prepare for the back propagation of pytorch
                baseline, C0,x = rectangle_process(temp,idx,Y,C0, mask,k,B,
                                                   Not_choosen_ones,Not_last_time,mask_list_num_with_lastone)
                C += baseline
        ###################
        # Loss and backprop handling 
        ###################
        
        loss = torch.mean((R - C) * logprobs)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print("epoch:{}, batch:{}/{}, reward:{}".format(epoch, i, steps,R.mean().item()))
            record_actor.append(R.mean().tolist())
            record_critic.append(C.tolist())
            plt.plot(record_actor,'r:')
            plt.plot(record_critic,'b:')
        if i % 100 == 0:
            print("record the last path to gazebo for showing up")
            #starting to show the path on simulated enviroment of cnc_machine 
            the_resent_path = temp.zig_zag_path(path_gazebo,mask_list_num_with_lastone)
            data = {'path':the_resent_path}
            data_1 = {'corners':path_gazebo}
            pathpoints_dir = os.path.join("pathpoints")
            if not os.path.exists(pathpoints_dir):
                os.makedirs(pathpoints_dir)
            name = 'pathpoints/path_points '+str(i)+'.yaml'
            with open(name, 'w') as file:
                documents = yaml.dump(data,file)
                documents = yaml.dump(data_1,file)
            path_gazebo = []
    time_one_epoch = time.time() - start #recording the work time of one epoch
    time_tot = time.time() - start_training_time + tot_time_ckpt
    ###################
    # Evaluate train model and baseline 
    # in this phase we just solve random instances with the actor and the critic
    # compare this soluation if we get any improvment we'll transfer the actor's
    # weights into the critic
    ###################
    # putting the actor in the eval mode
    Actor.eval()
    
    mean_tour_length_actor = 0
    mean_tour_length_critic = 0

    for step in range(0,B_valLoop):
        
        # compute tour for model and baseline
        # mask some points that are not the first visited points
        R = torch.zeros(B).cuda()
        logprobs = 0
        reward = torch.zeros(B).cuda()
        mask = torch.zeros(B,size).cuda() # use mask to make some points impossible to choose
        mask_temp = torch.zeros(B,size).cuda() # mask_temp for temp-masked points(wait for drying)
        mask_temp += -np.inf
        for initial_area_idx in mask_list_num:
            mask_temp[zero_to_bsz,initial_area_idx*4] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +1] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +2] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +3] = 0
        Not_choosen_ones = torch.ones(B,size).cuda()
        x = torch.zeros(B,dimension)# Y[:,0,:] #set the first point to x
        h = None
        c = None
        context = None        # set Y_ini to the out corner
        Transcontext = None
        Y0 = None
        waiting_time_list = waiting_time_initial

        with torch.no_grad():
            for k in range(size_rec):
                #add the time passed to the time axis of nessesary points
                reward_reshape = reward.view(B,1)
                waiting_time_list[zero_to_bsz] += Not_choosen_ones[zero_to_bsz] * reward_reshape
                Y = X.view(B,size,dimension)
                #refresh the mask list, fits the present situation
                while(True):
                    mask_temp = unmask_checker(R,B, waiting_time_list, mask_temp)
                    #check if all the points to be masked,if so, raise the total reward R, and check again
                    test_mask_tool = torch.zeros(B,size).cuda()
                    test_mask_tool += mask
                    test_mask_tool += mask_temp
                    test_output = F.softmax(test_mask_tool, dim=1)
                    test_anyone_open = torch.sum(test_output,1)
                    test_anyone_open = test_anyone_open>0
                    test_no_one_open = torch.logical_not(test_anyone_open)
                    if torch.sum(test_no_one_open.type(torch.uint8)) >0:
                        R+=test_no_one_open*(speed_of_nozzle*range_of_wait)
                        continue
                    break
                #same as the above part
                context, Transcontext, output, h, c, _ = Actor(context,Transcontext,x=x, 
                                                               X_all=X,h=h, c=c, mask=mask,mask_temp=mask_temp)
                idx = torch.argmax(output, dim=1)
                # prepare for the back propagation of pytorch
                reward, Y0,x = rectangle_process(temp, idx,Y,Y0, mask,k,B,
                                                 Not_choosen_ones,Not_last_time,mask_list_num_with_lastone)
                R += reward
        # critic baseline
        mask = torch.zeros(B,size).cuda() # use mask to make some points impossible to choose
        C = torch.zeros(B).cuda()
        baseline = torch.zeros(B).cuda()
        mask_temp = torch.zeros(B,size).cuda() # mask_temp for temp-masked points(wait for drying)
        mask_temp += -np.inf
        for initial_area_idx in mask_list_num:
            mask_temp[zero_to_bsz,initial_area_idx*4] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +1] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +2] = 0
            mask_temp[zero_to_bsz,initial_area_idx*4 +3] = 0
        Not_choosen_ones = torch.ones(B,size).cuda()
        x = torch.zeros(B,dimension)#Y[:,0,:] #set the first point to x
        h = None
        c = None
        context = None
        Transcontext = None
        C0 = None
        waiting_time_list = waiting_time_initial
        
        with torch.no_grad():
            for k in range(size_rec):
                #same as the above part
                baseline_reshape = baseline.view(B,1)
                waiting_time_list[zero_to_bsz] += Not_choosen_ones[zero_to_bsz] * baseline_reshape
                Y = X.view(B,size,dimension)
                while(True):
                    mask_temp = unmask_checker(C,B, waiting_time_list, mask_temp)
                    #check if all the points to be masked,if so, raise the total reward R, and check again
                    test_mask_tool = torch.zeros(B,size).cuda()
                    test_mask_tool += mask
                    test_mask_tool += mask_temp
                    test_output = F.softmax(test_mask_tool, dim=1)
                    test_anyone_open = torch.sum(test_output,1)
                    test_anyone_open = test_anyone_open>0
                    test_no_one_open = torch.logical_not(test_anyone_open)
                    if torch.sum(test_no_one_open.type(torch.uint8)) >0:
                        C+=test_no_one_open*(speed_of_nozzle*range_of_wait)
                        continue
                    break
                context, Transcontext, output, h, c, _ = Critic(context,Transcontext,x=x, X_all=X,h=h, c=c, mask=mask,mask_temp=mask_temp)
                idx = torch.argmax(output, dim=1)  
                # prepare for the back propagation of pytorch
                baseline, C0,x = rectangle_process(temp,idx,Y,C0, mask,k,B,
                                                   Not_choosen_ones,Not_last_time,mask_list_num_with_lastone)
                C += baseline

        mean_tour_length_actor  += R.mean().item()
        mean_tour_length_critic += C.mean().item()

    mean_tour_length_actor  =  mean_tour_length_actor  / B_valLoop
    mean_tour_length_critic =  mean_tour_length_critic / B_valLoop
    # evaluate train model and baseline and update if train model is better

    update_baseline = mean_tour_length_actor + TOL < mean_tour_length_critic

    print('Avg Actor {} --- Avg Critic {}'.format(mean_tour_length_actor,mean_tour_length_critic))

    if update_baseline:
        Critic.load_state_dict(Actor.state_dict())
        print('My actor is going on the right road Hallelujah :) Updated')
    ###################
    # Valdiation train model and baseline on 1k random TSP instances
    ###################
    # erased by daniel due to the 1K tsp is not the scale I want to train  

    # For checkpoint
    plot_performance_train.append([(epoch+1), mean_tour_length_actor])
    plot_performance_baseline.append([(epoch+1), mean_tour_length_critic])
    # compute the optimally gap ==> this is interesting because there is no LKH or other optimal algorithms 
    # for the problem like this rectangle characterized map
    mystring_min = 'Epoch: {:d}, epoch time: {:.3f}min, tot time: {:.3f}day, L_actor: {:.3f}, L_critic: {:.3f}, update: {}'.format(
        epoch, time_one_epoch/60, time_tot/86400, mean_tour_length_actor, mean_tour_length_critic, update_baseline)

    print(mystring_min)
    print('Save Checkpoints')

    # Saving checkpoint
    checkpoint_dir = os.path.join("checkpoint")

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    torch.save({
        'epoch': epoch,
        'time': time_one_epoch,
        'tot_time': time_tot,
        'loss': loss.item(),
        'plot_performance_train': plot_performance_train,
        'plot_performance_baseline': plot_performance_baseline,
        'model_baseline': Critic.state_dict(),
        'model_train': Actor.state_dict(),
        'optimizer': optimizer.state_dict(),
        },'{}.pkl'.format(checkpoint_dir + "/checkpoint_" + time_stamp + "-n{}".format(size) + "-gpu{}".format(gpu_id)))


  




                
            
                
        
                        

prepare to train
Hyper parameters:
learning rate 0.0001
batch size 128
steps 500
epoch 100
here
here
36
tensor([[  0.,   0.,   0.,   0.],
        [180., 180., 180., 180.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [180., 180., 180., 180.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [270., 270., 270., 270.],
        [  0.,   0.,   0.,   0.],
        [150., 150., 150., 150.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [ 30.,  30.,  30.,  30.],
        [  0.,   0.,   0.,   0.],
        [ 90.,  90.,  90.,  90.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [270., 270., 270., 270.],
        [  0.,   0.,   0.,   0.],
        [210., 210., 210., 210.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0

tensor([[ 981.5209,  981.5209,  981.5209,  981.5209],
        [1161.5209, 1161.5209, 1161.5209, 1161.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [1161.5209, 1161.5209, 1161.5209, 1161.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [1251.5209, 1251.5209, 1251.5209, 1251.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [1131.5209, 1131.5209, 1131.5209, 1131.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [1011.5209, 1011.5209, 1011.5209, 1011.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [1071.5209, 1071.5209, 1071.5209, 1071.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [ 981.5209,  981.5209,  981.5209,  981.5209],
        [1251.5209, 1251.520

tensor([[4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4312.9551, 4312.9551, 4312.9551, 4312.9551],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4312.9551, 4312.9551, 4312.9551, 4312.9551],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [1633.4982, 1633.4982, 1633.4982, 1633.4982],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [2740.2737, 2740.2737, 2740.2737, 2740.2737],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4162.9546, 4162.9546, 4162.9546, 4162.9546],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4222.9551, 4222.9551, 4222.9551, 4222.9551],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4132.9546, 4132.9546, 4132.9546, 4132.9546],
        [4402.9551, 4402.955

tensor([[7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [6528.2495, 6528.2495, 6528.2495, 6528.2495],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7458.9590, 7458.9590, 7458.9590, 7458.9590],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [1633.4982, 1633.4982, 1633.4982, 1633.4982],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [2740.2737, 2740.2737, 2740.2737, 2740.2737],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7308.9580, 7308.9580, 7308.9580, 7308.9580],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [5494.4873, 5494.4873, 5494.4873, 5494.4873],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [7278.9580, 7278.9580, 7278.9580, 7278.9580],
        [5287.7886, 5287.788

tensor([[19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [ 6528.2495,  6528.2495,  6528.2495,  6528.2495],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [17440.4570, 17440.4570, 17440.4570, 17440.4570],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [ 3497.2812,  3497.2812,  3497.2812,  3497.2812],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [ 5350.0796,  5350.0796,  5350.0796,  5350.0796],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [15871.0859, 15871.0859, 15871.0859, 15871.0859],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [ 9996.9736,  9996.9736,  9996.9736,  9996.9736],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [19520.8789, 19520.8789, 19520.8789, 19520.8789],
        [19520

...

tensor([[50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [13606.3096, 13606.3096, 13606.3096, 13606.3096],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [39786.6406, 39786.6406, 39786.6406, 39786.6406],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [12163.3125, 12163.3125, 12163.3125, 12163.3125],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [ 8239.1865,  8239.1865,  8239.1865,  8239.1865],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [36262.4336, 36262.4336, 36262.4336, 36262.4336],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [17329.2520, 17329.2520, 17329.2520, 17329.2520],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [50995.7695, 50995.7695, 50995.7695, 50995.7695],
        [50995

tensor([[61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [13606.3096, 13606.3096, 13606.3096, 13606.3096],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [46789.3672, 46789.3672, 46789.3672, 46789.3672],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [14027.0967, 14027.0967, 14027.0967, 14027.0967],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [10848.9932, 10848.9932, 10848.9932, 10848.9932],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [43163.3164, 43163.3164, 43163.3164, 43163.3164],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [21831.7383, 21831.7383, 21831.7383, 21831.7383],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [61986.2188, 61986.2188, 61986.2188, 61986.2188],
        [61986

tensor([[64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [16611.3262, 16611.3262, 16611.3262, 16611.3262],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [49794.3906, 49794.3906, 49794.3906, 49794.3906],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [15587.8857, 15587.8857, 15587.8857, 15587.8857],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [13226.5557, 13226.5557, 13226.5557, 13226.5557],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [45874.0469, 45874.0469, 45874.0469, 45874.0469],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [23766.0020, 23766.0020, 23766.0020, 23766.0020],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [64991.2422, 64991.2422, 64991.2422, 64991.2422],
        [64991

tensor([[68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [17233.3828, 17233.3828, 17233.3828, 17233.3828],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [52203.0742, 52203.0742, 52203.0742, 52203.0742],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [15587.8857, 15587.8857, 15587.8857, 15587.8857],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [13226.5557, 13226.5557, 13226.5557, 13226.5557],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [45874.0469, 45874.0469, 45874.0469, 45874.0469],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [23766.0020, 23766.0020, 23766.0020, 23766.0020],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [68227.8828, 68227.8828, 68227.8828, 68227.8828],
        [68227

tensor([[71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [17233.3828, 17233.3828, 17233.3828, 17233.3828],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [52203.0742, 52203.0742, 52203.0742, 52203.0742],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [15587.8857, 15587.8857, 15587.8857, 15587.8857],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [13226.5557, 13226.5557, 13226.5557, 13226.5557],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [45874.0469, 45874.0469, 45874.0469, 45874.0469],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [23766.0020, 23766.0020, 23766.0020, 23766.0020],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [71335.4219, 71335.4219, 71335.4219, 71335.4219],
        [71335

In [ ]:
import torch
index = int(9.34)
table = [[1,2],[3,4]]
table_1 = [[3,44],[5,6]]
table = torch.as_tensor(table)
table_1 = torch.as_tensor(table_1)
result = torch.maximum(table_1[:,0],table_1[:,1])
print(result)

In [5]:
import torch
list1 = [-1,-2,4,3,None]
list1.sort()
print(list1)




TypeError: '<' not supported between instances of 'NoneType' and 'int'